In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os, sys
PATH = os.path.abspath("")[:-3]

In [2]:

# SCRAPPEAR UN USUARIO
from test_user import get_user_data

USER = "ComicGirlAshley"
user_data = get_user_data(USER)
print(user_data)

[340, False, 26626, 97, 19968, 134, 6255, 4.637254901960785, 17.470588235294116, 271.45098039215685, 0.8921568627450981, 0.1568627450980392, 26529]


c:\Users\maria\Documents\Uni\ia orgs\bot_detector\.venv\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [3]:
# Se leen los datos generados con model.py\n",
df_data = pd.read_csv(PATH + "data/training_new.csv").sample(frac=1).reset_index(drop=True)
data = df_data.iloc[:, 1:]
print(data.head())

   factor_rep  verified  followersCount  friendsCount  tweetsCount  \
0         205         0            2346          3848         1894   
1         263         0              60           199           52   
2          42         0             179          3189         3949   
3         321         0             121          1050           12   
4        6885         0              26             1         3160   

   listedCount  mediaCount  avg_reply  avg_retweet  like_reply  avg_quote  \
0           11         335   0.422222     0.000000    0.711111      0.000   
1            0           2   0.650000     0.100000    2.925000      0.075   
2            4          55   0.121951     0.073171    0.731707      0.000   
3            0           1   0.000000     0.000000    0.571429      0.000   
4            8        3159   0.000000     0.019231    0.000000      0.000   

   hora_rep  followers_diff  clase  
0  0.222222           -2346    bot  
1  0.275000             -60    bot  
2  0.

In [55]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
header_names = list(data.columns.values)
for col in data.iloc[:,:-1]:
    data[col] = preprocessing.MinMaxScaler().fit_transform(np.array(data[col]).reshape(-1,1))
x_data = data.iloc[:,:-1].to_numpy()
y_data = data.iloc[:,-1].to_numpy()
x_train, x_test, y_train, y_test = train_test_split(
    x_data, y_data, stratify=y_data, test_size=0.33)

In [56]:
#CONVERTIRLA SALIDA A UN VECTOR DE 0 Y 1
from sklearn.preprocessing import LabelBinarizer

encoder=LabelBinarizer()
label = encoder.fit_transform(y_train)
y_train_transformed = np.hstack((label, 1 - label))
label = encoder.fit_transform(y_test)
y_test_transformed = np.hstack((label, 1 - label))
print(y_train[:5])
print(y_train_transformed[:5])

['human' 'bot' 'bot' 'human' 'human']
[[1 0]
 [0 1]
 [0 1]
 [1 0]
 [1 0]]


In [57]:
# COMPROBAR TIPO DE LOS DATOS
print(type(x_train))
print(type(y_train_transformed))
# COMPROBAR DIMENSIONES DE LOS DATOS
print(x_train.shape)
print(y_train.shape)
print(y_train_transformed.shape)
print(x_test.shape)
print(y_test.shape)
print(y_test_transformed.shape) 
# OBTENER DIMENSION DE LA ENTRADA Y NÚMERO DE SALIDAS
input_shape = (x_train.shape[1] ,)
num_clases = y_test_transformed.shape[1]

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(1616, 13)
(1616,)
(1616, 2)
(796, 13)
(796,)
(796, 2)


In [64]:
# Crear modelo usando sigmoide
from keras.models import Sequential
from keras.layers import Dense
input_shape= (x_train.shape[1] ,)
model = Sequential()
arch = [20, 20, 0, "sigmoid", 0.2]
model.add(Dense(arch[0], input_shape=input_shape, activation=arch[3]))
model.add(Dense(arch[1], input_shape=input_shape, activation=arch[3]))
#model.add(Dense(arch[2], input_shape=input_shape, activation=arch[3]))
model.add(Dense(2, activation='softmax'))
model.summary()

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_67 (Dense)            (None, 20)                280       
                                                                 
 dense_68 (Dense)            (None, 20)                420       
                                                                 
 dense_69 (Dense)            (None, 2)                 42        
                                                                 
Total params: 742
Trainable params: 742
Non-trainable params: 0
_________________________________________________________________


In [65]:
# CONFIGURAR MODELO Y ENTRENAMIENTO
from keras.callbacks import EarlyStopping
lr = 0.2
arch[4] = lr
epochs = 80000
early_stopping = EarlyStopping(monitor='val_loss', patience=1)
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=lr, momentum=0), metrics=['accuracy','mse'], loss='mean_squared_error')
historico = model.fit(x_train, y_train_transformed, epochs=epochs, verbose=1, validation_split=0.2,
    shuffle=False, validation_freq=1, batch_size=16, callbacks=[early_stopping])

Epoch 1/80000
81/81 [==============================] - 1s 3ms/step - loss: 0.2213 - accuracy: 0.6618 - mse: 0.2213 - val_loss: 0.2005 - val_accuracy: 0.7130 - val_mse: 0.2005
Epoch 2/80000
81/81 [==============================] - 0s 2ms/step - loss: 0.1970 - accuracy: 0.6974 - mse: 0.1970 - val_loss: 0.1949 - val_accuracy: 0.7191 - val_mse: 0.1949
Epoch 3/80000
81/81 [==============================] - 0s 2ms/step - loss: 0.1945 - accuracy: 0.6920 - mse: 0.1945 - val_loss: 0.1940 - val_accuracy: 0.6944 - val_mse: 0.1940
Epoch 4/80000
81/81 [==============================] - 0s 2ms/step - loss: 0.1938 - accuracy: 0.6889 - mse: 0.1938 - val_loss: 0.1932 - val_accuracy: 0.6944 - val_mse: 0.1932
Epoch 5/80000
81/81 [==============================] - 0s 2ms/step - loss: 0.1934 - accuracy: 0.6865 - mse: 0.1934 - val_loss: 0.1926 - val_accuracy: 0.6852 - val_mse: 0.1926
Epoch 6/80000
81/81 [==============================] - 0s 2ms/step - loss: 0.1931 - accuracy: 0.6858 - mse: 0.1931 - val_loss

In [66]:
col_names = ['arquitectura', 'loss', 'val_loss', 'accuracy', 'val_accuracy', 'epoch']
#dff = pd.DataFrame([], columns=col_names)
dff = pd.read_csv(PATH + 'data/vals2.csv')
final_epoch=len(historico.history['loss'])-1
print(final_epoch)
tloss = historico.history['loss'][final_epoch-1]
print(tloss)
vloss = historico.history['val_loss'][final_epoch-1]
print(vloss)
tacc = historico.history['accuracy'][final_epoch-1]
print(tacc)
vacc = historico.history['val_accuracy'][final_epoch-1]
print(vacc)
datos = pd.DataFrame([[arch, tloss, vloss, tacc, vacc, final_epoch]], columns=col_names)
datos = pd.concat([dff, datos], ignore_index=True)
datos.to_csv(PATH + 'data/vals2.csv', index = False)

19
0.1913202852010727
0.1871906965970993
0.6873065233230591
0.6944444179534912
